# Классификация изображений. Сверточные сети. Полносвязные сети с предобучением и дообучением.

Выполнили Вилюмсон Александр, Николаев Егор из группы 4136 

In [1]:
import os

base_dir = "archive/flowers"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

def count_images_per_class(directory):
    counts = {}
    if os.path.exists(directory):
        for cat in os.listdir(directory):
            category_path = os.path.join(directory, cat)
            if os.path.isdir(category_path):
                counts[cat] = len([img for img in os.listdir(category_path) if img.endswith(".jpg")])
    return counts

train_counts = count_images_per_class(train_dir)
val_counts = count_images_per_class(val_dir)

# Вывод результата
print("Количество изображений в выборке 'train':")
for category, count in train_counts.items():
    print(f"  {category}: {count}")

print("\nКоличество изображений в выборке 'val':")
for category, count in val_counts.items():
    print(f"  {category}: {count}")

Количество изображений в выборке 'train':
  daisy: 611
  dandelion: 841
  rose: 627
  sunflower: 586
  tulip: 787

Количество изображений в выборке 'val':
  daisy: 153
  dandelion: 211
  rose: 157
  sunflower: 147
  tulip: 197


In [2]:
import pandas as pd

# Путь к директориям train и val
base_dir = "archive/flowers"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

# Функция для создания DataFrame из указанной директории
def create_dataframe(directory, type):
    data = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            for img in os.listdir(category_path):
                if img.endswith(".jpg"):
                    data.append({
                        "file_path": os.path.join(category_path, img),
                        "class": category,
                        "dataset_type": type,
                    })
    return pd.DataFrame(data)

train_df = create_dataframe(train_dir, "train")
val_df = create_dataframe(val_dir, "val")
data_df = pd.concat([train_df, val_df], ignore_index=True)

In [3]:
data_df['class'].value_counts()

class
dandelion    1052
tulip         984
rose          784
daisy         764
sunflower     733
Name: count, dtype: int64

In [4]:
from PIL import Image
import numpy as np

# Задаем целевой размер
TARGET_SIZE = (224, 224)

# Функция для преобразования изображения
def preprocess_image(file_path, target_size=TARGET_SIZE):
    with Image.open(file_path) as img:
        img = img.resize(target_size)
        img = img.convert('RGB')    
        img_array = np.array(img)   
        return img_array

data_df['processed_image'] = data_df['file_path'].apply(preprocess_image)
print(data_df['processed_image'].iloc[0].shape)

(224, 224, 3)


In [5]:
data_df['normalized_image'] = data_df['processed_image'].apply(lambda x: x / 255.0)
data_df.head(2)

,file_path,class,dataset_type,processed_image,normalized_image
0,archive/flowers\train\daisy\100080576_f52e8ee0...,daisy,train,"[[[136, 136, 134], [140, 140, 140], [145, 145,...","[[[0.5333333333333333, 0.5333333333333333, 0.5..."
1,archive/flowers\train\daisy\10140303196_b88d3d...,daisy,train,"[[[212, 215, 223], [208, 213, 219], [210, 212,...","[[[0.8313725490196079, 0.8431372549019608, 0.8..."
2,archive/flowers\train\daisy\10172567486_274882...,daisy,train,"[[[26, 15, 81], [26, 16, 80], [26, 16, 79], [2...","[[[0.10196078431372549, 0.058823529411764705, ..."
3,archive/flowers\train\daisy\10172636503_21bede...,daisy,train,"[[[30, 49, 30], [28, 43, 31], [24, 33, 29], [2...","[[[0.11764705882352941, 0.19215686274509805, 0..."
4,archive/flowers\train\daisy\102841525_bd6628ae...,daisy,train,"[[[146, 134, 98], [160, 154, 126], [177, 177, ...","[[[0.5725490196078431, 0.5254901960784314, 0.3..."


In [6]:
# Разделяем данные
train_data = data_df[data_df['dataset_type'] == 'train']
val_data = data_df[data_df['dataset_type'] == 'val']

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data['class'])
y_val = label_encoder.transform(val_data['class'])

X_train = np.stack(train_data['normalized_image'].values)
X_val = np.stack(val_data['normalized_image'].values)

X_train

array([[[[0.53333333, 0.53333333, 0.5254902 ],
         [0.54901961, 0.54901961, 0.54901961],
         [0.56862745, 0.56862745, 0.56862745],
         ...,
         [0.60392157, 0.60392157, 0.59607843],
         [0.60784314, 0.60784314, 0.60784314],
         [0.59215686, 0.59215686, 0.59215686]],

        [[0.52941176, 0.52941176, 0.52156863],
         [0.54509804, 0.54509804, 0.5372549 ],
         [0.56470588, 0.56470588, 0.56470588],
         ...,
         [0.60392157, 0.60392157, 0.59607843],
         [0.60392157, 0.60392157, 0.60392157],
         [0.59215686, 0.59215686, 0.59215686]],

        [[0.52156863, 0.5254902 , 0.51764706],
         [0.54509804, 0.54509804, 0.5372549 ],
         [0.56470588, 0.56470588, 0.56078431],
         ...,
         [0.6       , 0.6       , 0.59607843],
         [0.6       , 0.6       , 0.6       ],
         [0.58039216, 0.58039216, 0.58039216]],

        ...,

        [[0.17647059, 0.18823529, 0.10588235],
         [0.17254902, 0.18431373, 0.10196078]